In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="u2hfMfcVXYMwoR8wQ57e")
project = rf.workspace("cuongdm").project("financial-dataset")
version = project.version(12)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to financial-dataset-12 in yolov8:: 100%|██████████| 1212/1212 [00:00<00:00, 2267.55it/s]


In [ ]:
!pip install -U ultralytics opencv-python matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 38.4 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0


In [ ]:
from ultralytics import YOLO
import os
import glob
import yaml
import cv2
import matplotlib.pyplot as plt
from collections import Counter


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
DATASET_DIR = "financial-dataset-12"
DATA_YAML = os.path.join(DATASET_DIR, "data.yaml")

with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)

class_names = data_cfg["names"]
num_classes = len(class_names)

print("Number of classes:", num_classes)
print("Class names:")
for i, name in enumerate(class_names):
    print(f"  {i}: {name}")


Number of classes: 3
Class names:
  0: 0
  1: page
  2: title


In [ ]:
def count_images(split):
    return len(glob.glob(os.path.join(DATASET_DIR, split, "images", "*")))

for split in ["train", "valid", "test"]:
    print(f"{split}: {count_images(split)} images")


train: 457 images
valid: 73 images
test: 70 images


In [ ]:
label_counter = Counter()

for split in ["train", "valid", "test"]:
    label_files = glob.glob(os.path.join(DATASET_DIR, split, "labels", "*.txt"))
    for lf in label_files:
        with open(lf, "r") as f:
            for line in f:
                cls_id = int(line.strip().split()[0])
                label_counter[cls_id] += 1

print("Label distribution:")
for cls_id, count in label_counter.items():
    print(f"{cls_id} ({class_names[cls_id]}): {count}")


Label distribution:
0 (0): 982
1 (page): 575
2 (title): 547


In [ ]:
model = YOLO("yolov8n.pt")  # nhẹ, train nhanh


In [ ]:
model.train(
    data=DATA_YAML,
    epochs=40,
    imgsz=640,
    batch=32,
    device=0,        # GPU = 0, CPU = 'cpu'
    workers=8,
    project="runs",
    name="yolov8-financial",
    save=True
)


Ultralytics 8.3.251 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=financial-dataset-12/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8-financial, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=Tr

KeyboardInterrupt: 

In [ ]:
best_model = YOLO("/content/best (4).pt")

metrics = best_model.val(
    data=DATA_YAML,
    split="test",
    imgsz=640,
    batch=32
)

print(metrics)


In [ ]:
TEST_IMG_DIR = os.path.join(DATASET_DIR, "test", "images")

results = best_model.predict(
    source=TEST_IMG_DIR,
    imgsz=640,
    conf=0.25,
    save=False
)


In [ ]:
def plot_result(result):
    img = result.orig_img.copy()

    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])

        label = f"{class_names[cls_id]} {conf:.2f}"
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(
            img, label, (x1, y1-5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5,
            (0,255,0), 2
        )

    plt.figure(figsize=(8,8))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()


In [ ]:
for i, r in enumerate(results):
    if i == 5:  # xem 5 ảnh
        break
    plot_result(r)


In [ ]:
!pip install -U ultralytics pdf2image opencv-python pillow matplotlib
!apt-get install -y poppler-utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.8 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfully uninstalled pillow-11.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.1.0 which is incompatible.


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 2s (116 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# from pdf2image import convert_from_path
# import os

# PDF_PATH = "/content/BOT_000000014944415_VI_Baocaotaichinh_kiemtoan_2024.pdf"
# OUT_IMG_DIR = "/content/pdf_pages"

# os.makedirs(OUT_IMG_DIR, exist_ok=True)

# pages = convert_from_path(PDF_PATH, dpi=300)

# img_paths = []
# for i, page in enumerate(pages):
#     out_path = os.path.join(OUT_IMG_DIR, f"page_{i:03d}.jpg")
#     page.save(out_path, "JPEG")
#     img_paths.append(out_path)

# print(f"Extracted {len(img_paths)} pages")


In [ ]:
from ultralytics import YOLO

MODEL_PATH = "/content/best (4).pt"
model = YOLO(MODEL_PATH)

CLASS_NAMES = model.names
print(CLASS_NAMES)


{0: '0', 1: 'page', 2: 'title'}


In [ ]:
import zipfile
import os

zip_path = '/content/pdf.zip'
extract_dir = '/content/extracted_pdf'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f'Successfully extracted {zip_path} to {extract_dir}')

Successfully extracted /content/pdf.zip to /content/extracted_pdf


In [ ]:
# import cv2
# import matplotlib.pyplot as plt
# import numpy as np

# OUT_VIS_DIR = "/content/yolo_vis"
# os.makedirs(OUT_VIS_DIR, exist_ok=True)

# def draw_boxes_and_contours(img, result):
#     vis = img.copy()

#     for box in result.boxes:
#         x1, y1, x2, y2 = map(int, box.xyxy[0])
#         cls_id = int(box.cls[0])
#         conf = float(box.conf[0])

#         # Bounding box
#         cv2.rectangle(vis, (x1,y1), (x2,y2), (0,255,0), 2)

#         label = f"{CLASS_NAMES[cls_id]} {conf:.2f}"
#         cv2.putText(
#             vis, label, (x1, y1-5),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.5,
#             (0,255,0), 2
#         )

#         # Contour (box -> polygon)
#         contour = np.array([
#             [x1,y1],
#             [x2,y1],
#             [x2,y2],
#             [x1,y2]
#         ])
#         cv2.drawContours(vis, [contour], -1, (255,0,0), 2)

#     return vis


In [ ]:
# results = model.predict(
#     source=OUT_IMG_DIR,
#     imgsz=640,
#     conf=0.25,
#     iou=0.3,
#     save=False
# )

# for img_path, res in zip(sorted(img_paths), results):
#     img = cv2.imread(img_path)
#     vis = draw_boxes_and_contours(img, res)

#     out_path = os.path.join(
#         OUT_VIS_DIR,
#         os.path.basename(img_path)
#     )
#     cv2.imwrite(out_path, vis)


In [ ]:
# sample = sorted(os.listdir(OUT_VIS_DIR))[:3]

# for f in sample:
#     img = cv2.imread(os.path.join(OUT_VIS_DIR, f))
#     plt.figure(figsize=(8,10))
#     plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#     plt.title(f)
#     plt.axis("off")
#     plt.show()


In [ ]:
!apt-get install -y tesseract-ocr
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
pip install -U transformers accelerate einops


BẮT ĐẦU LUỒNG Ở CELL NÀY

In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
from transformers import AutoModel, AutoTokenizer
from torchvision.transforms.functional import InterpolationMode
import os

# =========================
# CONSTANTS
# =========================
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
DEFAULT_IMAGE_SIZE = 448

# =========================
# TRANSFORM
# =========================
def build_transform(input_size=DEFAULT_IMAGE_SIZE):
    return T.Compose([
        T.Lambda(lambda img: img.convert("RGB")),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ])

# =========================
# DYNAMIC PREPROCESS
# Trả về list ảnh đã resize
# =========================
def dynamic_preprocess(image, image_size=DEFAULT_IMAGE_SIZE):
    return [image.resize((image_size, image_size))]  # hiện tại 1 ảnh, dễ mở rộng batch

# =========================
# VINTERN OCR
# =========================
def vintern_ocr_image(model, tokenizer, image, device="cuda", save_dir="crops"):
    os.makedirs(save_dir, exist_ok=True)

    if not hasattr(vintern_ocr_image, "counter"):
        vintern_ocr_image.counter = 1

    # -------------------------
    # PREPROCESS
    # -------------------------
    images = dynamic_preprocess(image)  # list ảnh resized

    # -------------------------
    # SAVE CROP (RESIZED)
    # -------------------------
    crop_name = f"crop_{vintern_ocr_image.counter}.png"
    images[0].save(os.path.join(save_dir, crop_name))
    vintern_ocr_image.counter += 1

    # -------------------------
    # TRANSFORM VÀ CHUYỂN SANG DEVICE
    # -------------------------
    transform = build_transform(DEFAULT_IMAGE_SIZE)
    pixel_values = torch.stack([transform(img) for img in images]).to(torch.bfloat16).to(device)

    # -------------------------
    # PROMPT CHO MODEL
    # -------------------------
    prompt = (
        "<image>\n"
        "Ảnh trên chứa một tiêu đề. "
        "Hãy xác định tiêu đề đó CÓ PHẢI là **Bảng Cân Đối Kế Toán** hay không. "
        "Điều kiện BẮT BUỘC: trong tiêu đề phải có từ **'kế toán'** (không phân biệt hoa/thường). "
        "Nếu thỏa mãn thì trả lời **yes**, nếu không thì trả lời **no**. "
        "Chỉ trả lời đúng một từ: yes hoặc no."
    )


    generation_config = dict(
        max_new_tokens=128,
        do_sample=False,
        num_beams=3
    )

    # -------------------------
    # GỌI MODEL
    # -------------------------
    response, _ = model.chat(
        tokenizer,
        pixel_values,
        prompt,
        generation_config,
        history=None,
        return_history=True
    )

    return response.strip()


In [ ]:
from transformers import AutoModel, AutoTokenizer

vintern_model = AutoModel.from_pretrained(
    "5CD-AI/Vintern-1B-v3_5",
    dtype=torch.bfloat16,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    use_flash_attn=False
).eval().cuda()

vintern_tokenizer = AutoTokenizer.from_pretrained(
    "5CD-AI/Vintern-1B-v3_5",
    trust_remote_code=True,
    use_fast=False
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


FlashAttention2 is not installed.


In [ ]:
# import os
# from pdf2image import convert_from_path
# from PIL import Image
# import pytesseract
# import re

# PDF_PATH = "/content/extracted_pdf/pdf/BOT_000000014944415_VI_Baocaotaichinh_kiemtoan_2024.pdf"

# # Lấy tên PDF (không đuôi .pdf) làm tên thư mục
# pdf_name = os.path.splitext(os.path.basename(PDF_PATH))[0]
# PAGE_IMG_DIR = f"/content/pdf_extractor/{pdf_name}"
# os.makedirs(PAGE_IMG_DIR, exist_ok=True)


# def fix_orientation(pil_img):
#     """
#     Detect & fix image orientation using Tesseract OSD
#     Returns corrected PIL image
#     """
#     try:
#         osd = pytesseract.image_to_osd(pil_img)
#         angle = int(re.search(r"Rotate: (\d+)", osd).group(1))

#         if angle != 0:
#             # Tesseract nói cần rotate bao nhiêu để đúng
#             pil_img = pil_img.rotate(-angle, expand=True)

#         return pil_img

#     except Exception as e:
#         # Nếu OSD fail thì giữ nguyên ảnh
#         print("OSD failed:", e)
#         return pil_img


# # Convert PDF → images
# pages = convert_from_path(PDF_PATH, dpi=300)

# page_imgs = []
# for i, p in enumerate(pages):
#     # Fix orientation
#     p_fixed = fix_orientation(p)

#     path = os.path.join(PAGE_IMG_DIR, f"page_{i:03d}.jpg")
#     p_fixed.save(path, "JPEG", quality=95)

#     page_imgs.append(path)

# print("Total pages:", len(page_imgs))
# print("Saved to:", PAGE_IMG_DIR)


In [ ]:
# yolo_results = model.predict(
#     source=PAGE_IMG_DIR,
#     imgsz=640,
#     conf=0.5,
#     iou=0.4,
#     save=False
# )


In [ ]:
# import os
# import cv2
# from PIL import Image

# # Lấy tên pdf (không .pdf)
# pdf_name = os.path.splitext(os.path.basename(PDF_PATH))[0]

# CROP_DIR = f"/content/title_crops/title_crops_{pdf_name}"
# os.makedirs(CROP_DIR, exist_ok=True)

# title_crops = []  # (crop_pil, page_idx, bbox)

# # --- Step 1: xác định page hợp lệ ---
# valid_pages = set()

# for page_idx, res in enumerate(yolo_results):
#     classes = {int(box.cls[0]) for box in res.boxes}
#     if 0 in classes and 2 in classes:
#         valid_pages.add(page_idx)

# print("Pages with class 0 & 2:", sorted(valid_pages))

# # --- Step 2: chỉ crop title ở page hợp lệ ---
# for page_idx, res in enumerate(yolo_results):
#     if page_idx not in valid_pages:
#         continue

#     img = res.orig_img
#     h, w = img.shape[:2]

#     for box in res.boxes:
#         cls_id = int(box.cls[0])
#         if cls_id != 2:
#             continue

#         x1, y1, x2, y2 = map(int, box.xyxy[0])

#         # Clamp an toàn
#         x1, y1 = max(0, x1), max(0, y1)
#         x2, y2 = min(w, x2), min(h, y2)

#         crop = img[y1:y2, x1:x2]
#         crop_pil = Image.fromarray(
#             cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
#         )

#         crop_name = f"page{page_idx:03d}_title_{len(title_crops):03d}.png"
#         crop_path = os.path.join(CROP_DIR, crop_name)
#         crop_pil.save(crop_path)

#         title_crops.append((crop_pil, page_idx, (x1, y1, x2, y2)))

# print("Total title crops:", len(title_crops))
# print("Saved to:", CROP_DIR)


In [ ]:
# import os

# # Lấy tên pdf (không .pdf)
# pdf_name = os.path.splitext(os.path.basename(PDF_PATH))[0]

# # Thư mục lưu input cho Vintern theo tên pdf
# VINTERN_INPUT_DIR = f"/content/vintern_inputs/vintern_inputs_{pdf_name}"
# os.makedirs(VINTERN_INPUT_DIR, exist_ok=True)

# results = []

# for i, (crop_pil, page_idx, bbox) in enumerate(title_crops):
#     print(f"\n--- Title {i} | Page {page_idx} ---")

#     response = vintern_ocr_image(
#         model=vintern_model,
#         tokenizer=vintern_tokenizer,
#         image=crop_pil,
#         device="cuda",
#         save_dir=VINTERN_INPUT_DIR
#     )

#     print("Vintern:", response)

#     results.append({
#         "page": page_idx,
#         "bbox": bbox,
#         "vintern_response": response
#     })

# print("Total OCR results:", len(results))
# print("Saved Vintern inputs to:", VINTERN_INPUT_DIR)


In [ ]:
import os
import cv2
import re
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
from pathlib import Path
import glob

# =====================================================
# CONFIG
# =====================================================
PDF_FOLDER = "/content/extracted_pdf/pdf"  # Thư mục chứa các file PDF

YES_KEYWORDS = ["yes", "có"]

# =====================================================
# HELPER FUNCTIONS
# =====================================================
def fix_orientation(pil_img):
    """Tự động xoay ảnh về đúng hướng"""
    try:
        osd = pytesseract.image_to_osd(pil_img)
        angle = int(re.search(r"Rotate: (\d+)", osd).group(1))
        if angle != 0:
            pil_img = pil_img.rotate(-angle, expand=True)
    except Exception as e:
        print(f"  ⚠️ OSD failed: {e}")
    return pil_img

def process_single_pdf(pdf_path, model, vintern_model, vintern_tokenizer, vintern_ocr_image):
    """Xử lý một file PDF"""

    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    print(f"\n{'='*60}")
    print(f"📄 Processing: {pdf_name}")
    print(f"{'='*60}")

    # Giữ nguyên format thư mục như code cũ
    PAGE_IMG_DIR   = f"/content/pdf_extractor/{pdf_name}"
    TITLE_CROP_DIR = f"/content/title_crops/{pdf_name}"
    TABLE_IMG_DIR  = f"/content/table_image/{pdf_name}"
    VINTERN_DIR    = f"/content/vintern_inputs/{pdf_name}"

    os.makedirs(PAGE_IMG_DIR, exist_ok=True)
    os.makedirs(TITLE_CROP_DIR, exist_ok=True)
    os.makedirs(TABLE_IMG_DIR, exist_ok=True)
    os.makedirs(VINTERN_DIR, exist_ok=True)

    try:
        # =====================================================
        # STEP 1: PDF → IMAGE
        # =====================================================
        print("\n[STEP 1] Converting PDF to images...")
        pages = convert_from_path(pdf_path, dpi=300)

        page_paths = []
        for i, p in enumerate(pages):
            p = fix_orientation(p)
            path = os.path.join(PAGE_IMG_DIR, f"page_{i:03d}.jpg")
            p.save(path, "JPEG", quality=95)
            page_paths.append(path)

        print(f"✅ Pages extracted: {len(page_paths)}")

        # =====================================================
        # STEP 2: YOLO DETECT
        # =====================================================
        print("\n[STEP 2] Running YOLO detection...")
        yolo_results = []

        for path in page_paths:
            img = cv2.imread(path)
            res = model(img)[0]
            res.orig_img = img
            yolo_results.append(res)

        # =====================================================
        # STEP 3: VALID PAGE (class 0 & 2)
        # =====================================================
        print("\n[STEP 3] Filtering valid pages...")
        valid_pages = set()

        for idx, res in enumerate(yolo_results):
            classes = {int(b.cls[0]) for b in res.boxes}
            if 0 in classes and 2 in classes:
                valid_pages.add(idx)

        print(f"✅ Valid pages: {sorted(valid_pages)}")

        # =====================================================
        # STEP 4: CROP TITLE (class = 2)
        # =====================================================
        print("\n[STEP 4] Cropping titles...")
        title_crops = []

        for page_idx in valid_pages:
            res = yolo_results[page_idx]
            img = res.orig_img
            h, w = img.shape[:2]

            for box in res.boxes:
                if int(box.cls[0]) != 2:
                    continue

                x1, y1, x2, y2 = map(int, box.xyxy[0])
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w, x2), min(h, y2)

                crop = img[y1:y2, x1:x2]
                crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

                name = f"page{page_idx:03d}_title_{len(title_crops):03d}.png"
                crop_pil.save(os.path.join(TITLE_CROP_DIR, name))

                title_crops.append((crop_pil, page_idx))

        print(f"✅ Title crops: {len(title_crops)}")

        # =====================================================
        # STEP 5: VINTERN + CROP TABLE
        # =====================================================
        print("\n[STEP 5] Processing with Vintern and extracting tables...")
        total_tables = 0

        for i, (crop_pil, page_idx) in enumerate(title_crops):
            print(f"\n--- TITLE {i} | PAGE {page_idx} ---")

            response = vintern_ocr_image(
                model=vintern_model,
                tokenizer=vintern_tokenizer,
                image=crop_pil,
                device="cuda",
                save_dir=VINTERN_DIR
            )

            print("Vintern:", response)

            if not isinstance(response, str):
                continue

            response = response.lower()
            if not any(k in response for k in YES_KEYWORDS):
                print("❌ Not table title")
                continue

            print("✅ YES detected → crop tables")

            res = yolo_results[page_idx]
            img = res.orig_img
            h, w = img.shape[:2]

            saved = 0
            for j, box in enumerate(res.boxes):
                if int(box.cls[0]) != 0:
                    continue

                x1, y1, x2, y2 = map(int, box.xyxy[0])
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w, x2), min(h, y2)

                crop = img[y1:y2, x1:x2]
                crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

                save_name = f"page{page_idx:03d}_table_{saved:03d}.jpg"
                crop_pil.save(os.path.join(TABLE_IMG_DIR, save_name))
                print("   saved:", save_name)
                saved += 1

            total_tables += saved

        print(f"\n✅ PDF complete! Tables extracted: {total_tables}")
        return True

    except Exception as e:
        print(f"\n❌ Error processing {pdf_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

# =====================================================
# MAIN EXECUTION
# =====================================================
def process_all_pdfs(model, vintern_model, vintern_tokenizer, vintern_ocr_image):
    """Xử lý toàn bộ PDF trong thư mục"""

    # Tìm tất cả file PDF
    pdf_files = sorted(glob.glob(os.path.join(PDF_FOLDER, "*.pdf")))

    if not pdf_files:
        print(f"❌ No PDF files found in {PDF_FOLDER}")
        return

    print(f"\n{'='*60}")
    print(f"🚀 Found {len(pdf_files)} PDF files to process")
    print(f"{'='*60}")

    success_count = 0
    fail_count = 0

    for idx, pdf_path in enumerate(pdf_files, 1):
        print(f"\n\n{'#'*60}")
        print(f"📊 Processing PDF {idx}/{len(pdf_files)}")
        print(f"{'#'*60}")

        result = process_single_pdf(
            pdf_path,
            model,
            vintern_model,
            vintern_tokenizer,
            vintern_ocr_image
        )

        if result:
            success_count += 1
        else:
            fail_count += 1

    # Summary
    print(f"\n\n{'='*60}")
    print(f"🎉 BATCH PROCESSING COMPLETE")
    print(f"{'='*60}")
    print(f"✅ Successfully processed: {success_count}/{len(pdf_files)}")
    print(f"❌ Failed: {fail_count}/{len(pdf_files)}")
    print(f"📁 Output directories:")
    print(f"   - /content/pdf_extractor/")
    print(f"   - /content/title_crops/")
    print(f"   - /content/table_image/")
    print(f"   - /content/vintern_inputs/")
    print(f"{'='*60}\n")

# =====================================================
# USAGE
# =====================================================
# Gọi hàm này với các model đã load:
process_all_pdfs(model, vintern_model, vintern_tokenizer, vintern_ocr_image)


🚀 Found 23 PDF files to process


############################################################
📊 Processing PDF 1/23
############################################################

📄 Processing: AMV_000000015013462_VN_BCTC_HN_nam_2024

[STEP 1] Converting PDF to images...
✅ Pages extracted: 55

[STEP 2] Running YOLO detection...

0: 640x480 1 title, 103.3ms
Speed: 25.8ms preprocess, 103.3ms inference, 42.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 0, 1 page, 8.0ms
Speed: 5.5ms preprocess, 8.0ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 0, 1 page, 1 title, 9.2ms
Speed: 4.5ms preprocess, 9.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 page, 1 title, 8.2ms
Speed: 4.7ms preprocess, 8.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 page, 1 title, 8.8ms
Speed: 4.2ms preprocess, 8.8ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 pa

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Vintern: no
❌ Not table title

--- TITLE 1 | PAGE 8 ---


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Vintern: Yes.
✅ YES detected → crop tables
   saved: page008_table_000.jpg

--- TITLE 2 | PAGE 9 ---


...

Vintern: Không.
❌ Not table title

--- TITLE 17 | PAGE 27 ---


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Vintern: No.
❌ Not table title

--- TITLE 18 | PAGE 28 ---


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Vintern: No.
❌ Not table title

✅ PDF complete! Tables extracted: 2


############################################################
📊 Processing PDF 19/23
############################################################

📄 Processing: VHG_000000014706961_VI_BaoCaoTaiChinh_KiemToan_2024.pdf

[STEP 1] Converting PDF to images...


In [ ]:
"""
/content/extracted_pdf/pdf/BOT_000000014944415_VI_Baocaotaichinh_kiemtoan_2024.pdf:  chỉ lấy được 1/3 ảnh
/content/extracted_pdf/pdf/BT6_000000014970920_VI_BaoCaoTaiChinh_KiemToan_2024.pdf lẩy đủ ảnh nhưng thừa 1 trang (do vintern phân loại sai tên title)
/content/extracted_pdf/pdf/VXP_000000014852285_CBTT_BCTC_kiem_toan_2024_dinh_kem.signed.pdf cũng ngon nhưng thừa 1 trang do ảnh titlt bị xoay ko được
/content/pdf_extractor/DCS_000000014779169_VI_BaoCaoTaiChinh_KiemToan__2022. ngon
/content/pdf_extractor/VNY_000000014707127_3.1_VI_VNY_Bao_cao_tai_chinh_hop_nhat_nam_2024da_nen ngon (2/2) nhưng thừa (do đoán bảng sai mặc dù ảnh không có bảng - nhưng ko nghiêm trọng vì nếu ko có thông tin thì chẳng extract được ở pha 2)
/content/pdf_extractor/VE1_000000014753067_VI_BCTC_nam_2024_da_kiem_toan_da_ky_VE1 đủ 1/1 ảnh
/content/pdf_extractor/CMC_000000014715923_VI_BaoCaoTaiChinh_KiemToan_2024. lấy được đủ 2/2 ảnh
/content/pdf_extractor/AMV_000000015013462_VN_BCTC_HN_nam_2024 (4/4) ảnh
/content/pdf_extractor/SBD_000000015054784_VI_BaoCaoTaiChinhKiemToan_Hop_Nhat_2024 đủ 2/2 ảnh
/content/pdf_extractor/SIG_000000014701703_Bao_cao_tai_chinh_hop_nhat_nam_2024__Signed_kem_giai_trinh: đủ 2/2 ảnh
/content/pdf_extractor/BVL_000000014725614_VI_BCTC_hop_nhat_2024.signed : lấy đủ 2/2 ảnh

"""

------------------

PHA 2

In [ ]:
import zipfile
import os

zip_path = '/content/table-transformer (1).zip'
extract_path = '/content/table-transformer'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"✅ Đã giải nén '{zip_path}' vào '{extract_path}'")

In [ ]:
import os

os.chdir("/content/table-transformer/src")
print("📂 Đã chuyển vào thư mục:", os.getcwd())


In [ ]:
!pip uninstall fitz
!pip install tools
# Cập nhật pip nếu cần
!pip install --upgrade pip

# Cài đặt các gói qua pip (chỉ chọn được gói tương đương từ environment.yml)
!pip install torch==1.13.1 torchvision==0.14.1 \
    pandas==1.5.3 scikit-learn==1.2.1 tqdm==4.65.0 \
    Cython==0.29.33 matplotlib==3.7.0 numpy==1.24.2 \
    pycocotools==2.0.6 Pillow==9.4.0 editdistance==0.6.2 \
    scikit-image==0.20.0 scipy==1.10.1 PyMuPDF==1.21.1

In [ ]:
%cd /content/table-transformer/src

In [ ]:
# !python predict.py \
#   --image_dir /content/table_image/VE1_000000014753067_VI_BCTC_nam_2024_da_kiem_toan_da_ky_VE1 \
#   --pdf_name VE1_000000014753067_VI_BCTC_nam_2024_da_kiem_toan_da_ky_VE1 \
#   --model_path "/content/model_20 (3).pth" \
#   --config_file structure_config.json


In [ ]:
!python visualize_table_structure.py \
  --pdf_name VE1_000000014753067_VI_BCTC_nam_2024_da_kiem_toan_da_ky_VE1


In [ ]:
pip install --force-reinstall pymupdf

In [ ]:
# !python predict.py \
#   --image_dir /content/table_image/CMI_000000014698416_VI_BaoCaoTaiChinh_KiemToan_2024 \
#   --pdf_name CMI_000000014698416_VI_BaoCaoTaiChinh_KiemToan_2024 \
#   --model_path "/content/model_20 (3).pth" \
#   --config_file structure_config.json \
#   --score_threshold 0.2 \
#   --device cuda


In [ ]:
!python predict.py \
    --config_file structure_config.json \
    --model_path "/content/model_20 (3).pth" \
    --input_root /content/table_image \
    --output_root /content/table_structure \
    --score_threshold 0.2

In [ ]:
!pip install paddleocr==2.10.0
!pip install paddlepaddle==2.6.2

In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
from transformers import AutoModel, AutoTokenizer
from torchvision.transforms.functional import InterpolationMode
import os

# =========================
# CONSTANTS
# =========================
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
DEFAULT_IMAGE_SIZE = 448

# =========================
# TRANSFORM
# =========================
def build_transform(input_size=DEFAULT_IMAGE_SIZE):
    return T.Compose([
        T.Lambda(lambda img: img.convert("RGB")),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ])

# =========================
# DYNAMIC PREPROCESS
# Trả về list ảnh đã resize
# =========================
def dynamic_preprocess(image, image_size=DEFAULT_IMAGE_SIZE):
    return [image.resize((image_size, image_size))]  # hiện tại 1 ảnh, dễ mở rộng batch

# =========================
# VINTERN OCR
# =========================
def vintern_ocr_image(model, tokenizer, image, device="cuda", save_dir="crops"):
    os.makedirs(save_dir, exist_ok=True)

    if not hasattr(vintern_ocr_image, "counter"):
        vintern_ocr_image.counter = 1

    # -------------------------
    # PREPROCESS
    # -------------------------
    images = dynamic_preprocess(image)  # list ảnh resized

    # -------------------------
    # SAVE CROP (RESIZED)
    # -------------------------
    crop_name = f"crop_{vintern_ocr_image.counter}.png"
    images[0].save(os.path.join(save_dir, crop_name))
    vintern_ocr_image.counter += 1

    # -------------------------
    # TRANSFORM VÀ CHUYỂN SANG DEVICE
    # -------------------------
    transform = build_transform(DEFAULT_IMAGE_SIZE)
    pixel_values = torch.stack([transform(img) for img in images]).to(torch.bfloat16).to(device)

    # -------------------------
    # PROMPT CHO MODEL
    # -------------------------
    prompt = (
        "<image>\n"
        "Chỉ trả về chính xác nội dung chữ thực sự nhìn thấy trong ảnh. "
        "Không giải thích, không suy đoán, không tự tạo nội dung. "
        "Nếu chữ bị mờ, nhiễu hoặc không chắc chắn thì trả về rỗng. "
        "Ưu tiên đọc đúng tiếng Việt có dấu. "
        "Khi nội dung là số, hãy đọc liền thành một chuỗi số và "
        "bỏ các dấu phân cách hàng nghìn hoặc hàng triệu như '.' hoặc ','. "
        "Ví dụ: '1.234.567' hoặc '1,234,567' → '1234567'."
    )

    generation_config = dict(
        max_new_tokens=128,
        do_sample=False,
        num_beams=3
    )

    # -------------------------
    # GỌI MODEL
    # -------------------------
    response, _ = model.chat(
        tokenizer,
        pixel_values,
        prompt,
        generation_config,
        history=None,
        return_history=True
    )

    return response.strip()


In [ ]:
# %cd /content/table-transformer/src

In [ ]:
import shutil
import os

output_zip_name = '/content/table-transformer'
directory_to_zip = '/content/table-transformer'

shutil.make_archive(output_zip_name, 'zip', directory_to_zip)

print(f'Successfully zipped {directory_to_zip} to {output_zip_name}.zip')

In [ ]:
vintern_model = AutoModel.from_pretrained(
    "5CD-AI/Vintern-1B-v3_5",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    use_flash_attn=False
).eval().cuda()

vintern_tokenizer = AutoTokenizer.from_pretrained(
    "5CD-AI/Vintern-1B-v3_5",
    trust_remote_code=True,
    use_fast=False
)


In [ ]:
# =========================
# TABLE RECONSTRUCTION PIPELINE (PaddleOCR v2.10 Safe)
# =========================

import os
import json
import cv2
import numpy as np
from PIL import Image
from paddleocr import PaddleOCR
from tqdm import tqdm

# =========================
# CONFIG
# =========================
DEBUG_DIR = "/content/table_structure/VE1_000000014753067_VI_BCTC_nam_2024_da_kiem_toan_da_ky_VE1"
IMAGE_DIR = "/content/table_image/VE1_000000014753067_VI_BCTC_nam_2024_da_kiem_toan_da_ky_VE1"
OUT_DIR = "/content/result_reconstruct/VE1_000000014753067_VI_BCTC_nam_2024_da_kiem_toan_da_ky_VE1"
os.makedirs(OUT_DIR, exist_ok=True)

TEXT_IN_CELL_THRESH = 0.3
IOU_OBJ_THRESH = 0.5

CLASS_MAP = {
    0: "table",
    1: "table column",
    2: "table row",
    3: "table column header",
    4: "table spanning cell",
    5: "table projected row header",
}

# =========================
# OCR INIT
# =========================
ocr = PaddleOCR(lang="en", use_angle_cls=True, rec=True, det=True)

# =========================
# UTILS
# =========================
def iou(a, b):
    xa, ya = max(a[0], b[0]), max(a[1], b[1])
    xb, yb = min(a[2], b[2]), min(a[3], b[3])
    inter = max(0, xb - xa) * max(0, yb - ya)
    if inter <= 0:
        return 0.0
    area_a = (a[2]-a[0])*(a[3]-a[1])
    area_b = (b[2]-b[0])*(b[3]-b[1])
    return inter / (area_a + area_b - inter)

def text_overlap_ratio(text_box, cell_box):
    xa, ya = max(text_box[0], cell_box[0]), max(text_box[1], cell_box[1])
    xb, yb = min(text_box[2], cell_box[2]), min(text_box[3], cell_box[3])
    inter = max(0, xb - xa) * max(0, yb - ya)
    if inter <= 0:
        return 0.0
    text_area = (text_box[2]-text_box[0])*(text_box[3]-text_box[1])
    if text_area <= 0:
        return 0.0
    return inter / text_area

def extract_ocr_boxes_only(ocr_result):
    """Chỉ lấy bounding box từ PaddleOCR v2.10"""
    boxes = []
    for line in ocr_result:
        # Nếu nested line (1 page chứa nhiều line)
        if isinstance(line[0], list) and isinstance(line[0][0], list):
            lines_to_process = line
        else:
            lines_to_process = [line]

        for item in lines_to_process:
            if len(item) == 3:
                bbox, text, conf = item
            elif len(item) == 2:
                bbox, text = item
                conf = 1.0
            else:
                continue
            xs = [int(pt[0]) for pt in bbox]
            ys = [int(pt[1]) for pt in bbox]
            boxes.append({"bbox": [min(xs), min(ys), max(xs), max(ys)], "text": text})
    return boxes

# =========================
# MAIN LOOP
# =========================
for jf in tqdm(sorted(os.listdir(DEBUG_DIR))):
    if not jf.endswith(".json"):
        continue

    with open(os.path.join(DEBUG_DIR, jf), "r", encoding="utf-8") as f:
        data = json.load(f)

    preds = data.get("predictions", [])
    if not preds:
        continue

    rows = [p for p in preds if p["label"] == 2]
    cols = [p for p in preds if p["label"] == 1]
    col_headers = [p for p in preds if p["label"] == 3]
    proj_headers = [p for p in preds if p["label"] == 5]
    spans = [p for p in preds if p["label"] == 4]

    if not rows or not cols:
        continue

    rows.sort(key=lambda x: x["bbox"][1])
    cols.sort(key=lambda x: x["bbox"][0])

    # -------------------------
    # BUILD GRID
    # -------------------------
    cells = []
    for r_id, row in enumerate(rows):
        rymin, rymax = row["bbox"][1], row["bbox"][3]
        row_cells = []
        for c_id, col in enumerate(cols):
            cxmin, cxmax = col["bbox"][0], col["bbox"][2]
            row_cells.append({
                "row_id": r_id,
                "col_id": c_id,
                "cell_id": f"r{r_id}_c{c_id}",
                "bbox": [cxmin, rymin, cxmax, rymax],
                "header": False,
                "projected": False,
                "spanning": False,
                "spanning_id": None,
                "texts": []
            })
        cells.append(row_cells)

    # -------------------------
    # MARK HEADERS / PROJECTED
    # -------------------------
    for r_id, row in enumerate(rows):
        for h in col_headers:
            if iou(row["bbox"], h["bbox"]) > IOU_OBJ_THRESH:
                for c in cells[r_id]:
                    c["header"] = True
        for p in proj_headers:
            if iou(row["bbox"], p["bbox"]) > IOU_OBJ_THRESH:
                for c in cells[r_id]:
                    c["projected"] = True

    # -------------------------
    # MARK SPANNING CELLS
    # -------------------------
    span_counter = 0
    for s in spans:
        span_id = f"span_{span_counter}"
        span_counter += 1
        for row in cells:
            for c in row:
                if iou(c["bbox"], s["bbox"]) > IOU_OBJ_THRESH:
                    c["spanning"] = True
                    c["spanning_id"] = span_id

    # -------------------------
    # LOAD IMAGE
    # -------------------------
    img_name = data.get("image")
    if not img_name:
        continue

    img_path = os.path.join(IMAGE_DIR, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    # -------------------------
    # OCR: PaddleOCR v2.10 → BBOX + TEXT
    # -------------------------
    ocr_res = ocr.ocr(img, cls=True)
    ocr_boxes = extract_ocr_boxes_only(ocr_res)

    # -------------------------
    # CROP → VINTERN OCR → ASSIGN CELL
    # -------------------------
    for o in ocr_boxes:
        x1, y1, x2, y2 = map(int, o["bbox"])
        crop = img[y1:y2, x1:x2]
        if crop.size == 0:
            continue
        crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

        try:
            vintern_text = vintern_ocr_image(
                vintern_model,
                vintern_tokenizer,
                crop_pil
            )
            print("Text: ", vintern_text)
        except Exception as e:
            print("❌ OCR failed:", e)
            continue

        vintern_text = vintern_text.strip()
        if not vintern_text:
            continue

        best_score = 0
        best_cell = None
        for row in cells:
            for c in row:
                score = text_overlap_ratio(o["bbox"], c["bbox"])
                if score > best_score:
                    best_score = score
                    best_cell = c

        if best_score >= TEXT_IN_CELL_THRESH and best_cell:
            best_cell["texts"].append(vintern_text)

    # -------------------------
    # REMOVE EMPTY ROWS
    # -------------------------
    new_cells = []
    new_row_id = 0
    for row in cells:
        if all(" ".join(c["texts"]).strip() == "" for c in row):
            continue
        new_row = []
        for new_col_id, c in enumerate(row):
            new_c = c.copy()
            new_c["row_id"] = new_row_id
            new_c["col_id"] = new_col_id
            new_c["cell_id"] = f"r{new_row_id}_c{new_col_id}"
            new_c["text"] = " ".join(c["texts"]).strip()
            new_row.append(new_c)
        new_cells.append(new_row)
        new_row_id += 1

    # -------------------------
    # SAVE RESULT
    # -------------------------
    out = {
        "image": img_name,
        "table": new_cells
    }
    out_path = os.path.join(OUT_DIR, jf.replace(".json", "_vintern.json"))
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(out, f, indent=2, ensure_ascii=False)

print("✅ DONE — Header / Span preserved, OCR by Vintern + PaddleOCR v2.10")


In [ ]:
# =========================
# JSON TABLE → HTML RENDER
# =========================

import os
import json
from tqdm import tqdm

# =========================
# CONFIG
# =========================
DEBUG_DIR = "/content/result_reconstruct/SBD_000000015054784_VI_BaoCaoTaiChinhKiemToan_Hop_Nhat_2024"
HTML_DIR  = "/content/html/SBD_000000015054784_VI_BaoCaoTaiChinhKiemToan_Hop_Nhat_2024"
os.makedirs(HTML_DIR, exist_ok=True)

# =========================
# HTML TEMPLATE
# =========================
HTML_TEMPLATE = """<!DOCTYPE html>
<html lang="vi">
<head>
<meta charset="utf-8">
<title>{title}</title>
<style>
    body {{
        font-family: Arial, Helvetica, sans-serif;
        padding: 20px;
        background: #ffffff;
    }}

    h3 {{
        margin-bottom: 10px;
    }}

    table {{
        border-collapse: collapse;
        margin: 10px 0;
        width: 100%;
    }}

    td {{
        border: 1px solid #333;
        padding: 6px 10px;
        min-width: 80px;
        vertical-align: top;
        font-size: 14px;
        color: #000;
    }}

    /* COLUMN / ROW HEADER */
    .header {{
        background: #dbeafe;
        font-weight: bold;
        color: #000;   /* ✅ chữ tiêu đề luôn màu đen */
        text-align: center;
    }}

    /* PROJECTED ROW HEADER */
    .projected {{
        background: #dcfce7;
        font-weight: bold;
        color: #000;
    }}

    /* SPANNING CELL */
    .spanning {{
        background: #fef9c3;
        color: #000;
    }}
</style>
</head>
<body>

<h3>{title}</h3>

{table_html}

</body>
</html>
"""

# =========================
# BUILD TABLE HTML
# =========================
def build_html_table(table):
    """
    table: List[List[cell_dict]]
    """
    html = ["<table>"]

    for row in table:
        html.append("  <tr>")
        for cell in row:
            classes = []

            if cell.get("header"):
                classes.append("header")
            if cell.get("projected"):
                classes.append("projected")
            if cell.get("spanning"):
                classes.append("spanning")

            class_attr = f' class="{" ".join(classes)}"' if classes else ""
            text = cell.get("text", "")
            text = text.replace("\n", "<br>").strip()

            html.append(f"    <td{class_attr}>{text}</td>")
        html.append("  </tr>")

    html.append("</table>")
    return "\n".join(html)

# =========================
# MAIN LOOP
# =========================
for jf in tqdm(sorted(os.listdir(DEBUG_DIR))):
    if not jf.endswith(".json"):
        continue

    json_path = os.path.join(DEBUG_DIR, jf)
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    table = data.get("table")
    if not table:
        continue

    table_html = build_html_table(table)

    title = jf.replace(".json", "")
    full_html = HTML_TEMPLATE.format(
        title=title,
        table_html=table_html
    )

    out_path = os.path.join(
        HTML_DIR,
        jf.replace(".json", ".html")
    )

    with open(out_path, "w", encoding="utf-8") as f:
        f.write(full_html)

print("✅ DONE! HTML files saved to:", HTML_DIR)


In [ ]:
from IPython.display import HTML, display

html_path = "/content/html/SBD_000000015054784_VI_BaoCaoTaiChinhKiemToan_Hop_Nhat_2024/page008_table_000_vintern.html"

with open(html_path, "r", encoding="utf-8") as f:
    html_content = f.read()

display(HTML(html_content))


In [ ]:
from IPython.display import HTML, display

html_path = "/content/html/CMI_000000014698416_VI_BaoCaoTaiChinh_KiemToan_202/page008_table_000_vintern.html"

with open(html_path, "r", encoding="utf-8") as f:
    html_content = f.read()

display(HTML(html_content))


In [ ]:
import shutil
import os

output_zip_name = '/content/html'
directory_to_zip = '/content/html'

shutil.make_archive(output_zip_name, 'zip', directory_to_zip)

print(f'Successfully zipped {directory_to_zip} to {output_zip_name}.zip')